In [26]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

- Bước đầu tiên để tính self-attention là tính ra các giá trị trung gian $ω$.
- $ω$ còn được gọi là attention scores (điểm chú ý).

In [32]:
query = inputs[1]   # x^2 (journey)
attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)

print("Attention scores:\n", attn_scores_2)


Attention scores:
 tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
inputs.shape[0]

6

In [7]:
attn_scores_2.sum()

tensor(6.5617)

- `divide by sum`

In [8]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- `softmax naive`

In [11]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- Hàm `softmax` giúp đầu ra luôn là `số dương` và tổng bằng 1, vì xác suất không thể âm hoặc vượt quá 1.

- Tuy nhiên hàm `softmax_naive` đơn giản ở trên có thể `bị tràn số` (overflow) hoặc `mất chính xác` (underflow) khi đầu vào `quá lớn` hoặc `quá nhỏ`.
    + Hàm mũ $e^x$ tăng rất nhanh, $x = 700$ thì $e^x \approx 10^{304}$
    + Tương tự với khi $x$ âm lớn thì $e^x$ về 0 rất nhanh, $x = -700$ thì $e^x \approx 1^{-306}$

- Vì vậy, ta sẽ dùng `torch.softmax` được tối ưu cho các vấn đề tính toán này.

In [31]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- `context vector`

In [13]:
query = inputs[1]   # x^2 (journey)
context_vector_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vector_2 += attn_weights_2[i] * x_i
print("Context vector:", context_vector_2)

Context vector: tensor([0.4419, 0.6515, 0.5683])


### Compute attention weights for all tokens

In [19]:
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print("Attention scores:\n", attn_scores)

Attention scores:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [42]:
attn_scores = inputs @ inputs.T
print("Attention scores:\n", attn_scores)

Attention scores:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- Sử dụng _matrix multiplication_ và _for_ đều cho kết quả như nhau nhưng _loop_ chậm hơn.

In [40]:
attn_weights = torch.softmax(attn_scores, dim=1)    # dim=1 để softmax theo hàng
print("Attention weights:\n", attn_weights)

Attention weights:
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [35]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

- Như ở trên ta đã tính _context vector_ cho token 2, ở đây ta sẽ _nhân ma trận_ để tính _context vector_ cho mọi input tokens.

In [43]:
all_context_vectors = attn_weights @ inputs
print("All context vectors:\n", all_context_vectors)

All context vectors:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- Ban đầu khi tạo `context vector` cho `token 2`, ta nhân từng `attention weights` với `input vector` tương ứng sau đó cộng hết 6 vector lại. 

- Thì ở bước nhân ma trận này, nhìn vào hàng 1 của `attention weights` ([0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452]) khi nhân với `ma trận inputs` thì từng phần tử của `attention weight 1` này sẽ nhân với từng `vector input` tương ứng, sau đó cũng cộng 6 vector lại để thành 1 vector mới có chiều `1x3` (và cũng là `context vector 1`). 

- Tương tự với 5 vector còn lại.